In [ ]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

from antra.antra import *
import json
from decode_abstract_models import *

In [ ]:
path_to_data = "../../../data-files/ReaSCAN-Simple/data-compositional-splits.txt"
data_json = json.load(open(path_to_data, "r"))

In [ ]:
NUM = 200
goods = []
for ex in data_json["examples"]["train"]:
    if ex["adverb_in_command"] == "":
        goods.append(ex)
        if len(goods) > NUM-1:
            break

In [ ]:
input = GraphInput({"examples":goods})
G = get_counter_compgraph()
_ = G.compute(input) # always return null?
for j in range(NUM):
    # print("\n\n\n", j, "a;oifwje;awfeoijawj \n\n\n")
    # print(goods[j]["command"])
    # print(goods[j]["target_commands"])
    for i in range(20):
        # print(G.compute_node("x" + str(i), input)[j])
        # print(G.compute_node("y" + str(i), input)[j])
        # print(G.compute_node("o" + str(i), input)[j])
        # print(G.compute_node("a" + str(i), input)[j])
        if len(goods[j]["target_commands"].split(",")) <= i and G.compute_node("a" + str(i), input)[j] != "NULL":
            # print("\n\nOH NO")
            # print(G.compute_node("a" + str(i), input)[j])
            # print(goods[j]["target_commands"].split(",")[i])
            print(MEME)
        if len(goods[j]["target_commands"].split(",")) > i and G.compute_node("a" + str(i), input)[j] != goods[j]["target_commands"].split(",")[i]:
            # print("\n\nOH NO")
            # print(G.compute_node("a" + str(i), input)[j])
            # print(goods[j]["target_commands"].split(",")[i])
            print(MEME)
    # print(G.compute_node("a" + str(i), input)[j] == "turn left")
    # print("\n")